In [10]:
import numpy as np
import trimesh
import tqdm 

In [11]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------

    vector_a = vertices[faces[:, 0]] - vertices[faces[:, 1]]
    vector_b = vertices[faces[:, 0]] - vertices[faces[:, 2]]
    area = np.linalg.norm(np.cross(vector_a, vector_b), axis=1) / 2.0

    prob = area / np.sum(area)
    sampled_faces = np.random.choice(faces.shape[0], size=sample_num, p=prob)
    sampled_faces = faces[sampled_faces]

    # use the alternative method in slides to sample points
    # x = (1 - sqrt(r1)) * v0 + sqrt(r1) * (1 - r2) * v1 + sqrt(r1) * r2 * v2
    # where r1, r2 are uniform random numbers in [0, 1]
    r1, r2 = np.random.rand(sample_num), np.random.rand(sample_num)

    uniform_pc = (
        (1 - np.sqrt(r1))[:, None] * vertices[sampled_faces[:, 0]]
        + (np.sqrt(r1) * (1 - r2))[:, None] * vertices[sampled_faces[:, 1]]
        + (np.sqrt(r1) * r2)[:, None] * vertices[sampled_faces[:, 2]]
    )

    return area, prob, uniform_pc
        

In [12]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------

	# init the distance with infinity
	distance = np.full((pc.shape[0],), np.inf)

	results = []
	# randomly sample the first point
	p = pc[np.random.randint(pc.shape[0])]
	results.append(p)

	for _ in tqdm.tqdm(range(sample_num - 1)):
		# update the distance
		distance = np.minimum(distance, np.linalg.norm(pc - p, axis=1))
		index = np.argmax(distance)
		results.append(pc[index])
		p = pc[index]

	results = np.array(results)
	return results

In [13]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [14]:
def save_point_cloud_to_obj(pc, filename):
    """
    Save point cloud to an .obj file.
    Args:
        pc: numpy array of shape (N, 3), where N is the number of points.
        filename: str, the output .obj file path.
    """
    with open(filename, "w") as f:
        for point in pc:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")
    print(f"Point cloud saved to {filename}")

save_point_cloud_to_obj(uniform_pc, "../results/uniform_pc.obj")


Point cloud saved to ../results/uniform_pc.obj


In [15]:
# task 2: FPS
init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

100%|██████████| 511/511 [00:00<00:00, 30874.68it/s]


In [16]:
save_point_cloud_to_obj(fps_pc, "../results/fps_pc.obj")

Point cloud saved to ../results/fps_pc.obj


The results of the point cloud sampling methods, which is visualized using MeshLab, are shown below. 
<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="../results/uniform_pc_visual.png" alt="Uniform Sampling" style="width: 100%;">
        <p style="text-align: center;">Uniform Sampling</p>
    </div>
    <div>
        <img src="../results/fps_pc_visual.png" alt="FPS Sampling" style="width: 100%;">
        <p style="text-align: center;">FPS Sampling</p>
    </div>
</div>

In [17]:
# task 3: metrics

from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD, EMD = np.zeros(5), np.zeros(5)
for i in range(5):
    # Uniform sampling
	_,_, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
	# FPS
	_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
	fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

	distance = uniform_pc[:, None] - fps_pc[None, :]

	cd_distance = np.mean(np.min(np.linalg.norm(distance, axis=2), axis=1)) + np.mean(np.min(np.linalg.norm(distance, axis=2), axis=0))
	cd_distance /= 2.0		# for fair comparison, we divide by 2

	# convert numpy array to list of tuples
	uniform_pc_list = [tuple(point) for point in uniform_pc]
	fps_pc_list = [tuple(point) for point in fps_pc]
	emd_distance = earthmover_distance(uniform_pc_list, fps_pc_list)
	CD[i], EMD[i] = cd_distance, emd_distance


CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

print(f"CD mean: {CD_mean}, CD var: {CD_var}")
print(f"EMD mean: {EMD_mean}, EMD var: {EMD_var}")

# # For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

100%|██████████| 511/511 [00:00<00:00, 20975.21it/s]


move 0.001953125 dirt from (30.934106581019766, 13.318456048210912, 26.98506684446722) to (31.137224869081845, 14.135144535106207, 27.03003019312716) for a cost of 0.0016460323212363558
move 0.001953125 dirt from (18.412215908835073, 40.79822309232952, 22.759432748609733) to (20.357348258083615, 36.98983867957989, 23.508755316266278) for a cost of 0.008479535769262901
move 0.001953125 dirt from (23.03554582504751, 19.90636097445518, 41.09900058310323) to (23.949866634633306, 19.196325387642332, 41.96729357234619) for a cost of 0.0028263451758202226
move 0.001953125 dirt from (25.540100641542264, 8.256528428216072, 35.348919111760054) to (26.001472020203575, 9.783298797053616, 34.31705522849459) for a cost of 0.003710235532803412
move 0.001953125 dirt from (44.85551188951558, 34.66836727584747, 32.442759959515186) to (46.7409662689016, 34.36251257062675, 31.486677154926696) for a cost of 0.004171913242433462
move 0.001953125 dirt from (15.389846876368896, 32.13284938198678, 25.019265987

100%|██████████| 511/511 [00:00<00:00, 36791.51it/s]


move 0.001953125 dirt from (15.776319598254066, 35.03329233757452, 22.630622255952634) to (14.169021175098212, 34.17576036019597, 22.67457916756909) for a cost of 0.003559139084274905
move 0.001953125 dirt from (50.121757090015514, 29.243922340628753, 38.093362096211024) to (49.3582117308692, 29.291772889052567, 38.71490338616495) for a cost of 0.0019251955855212955
move 0.001953125 dirt from (49.683396514407, 22.957509473555525, 22.62064482612019) to (49.76573135054997, 23.16434738227649, 22.677372676992196) for a cost of 0.00044870467694558
move 0.001953125 dirt from (12.461444025522255, 48.493224413237584, 26.479185271075682) to (12.238654025123758, 48.68343373304129, 27.356765601332327) for a cost of 0.001806996592540213
move 0.001953125 dirt from (31.798558711206866, 13.471822316596683, 38.99220973990139) to (31.725125443389018, 12.845463157888652, 38.74249836762604) for a cost of 0.001324780384740357
move 0.001953125 dirt from (7.4838827797556835, 33.5194805075204, 29.12157892384

100%|██████████| 511/511 [00:00<00:00, 19662.30it/s]


move 0.001953125 dirt from (30.57006946711673, 14.805271353631616, 28.711223733792128) to (28.942030420312605, 15.066816405521925, 28.932120359841992) for a cost of 0.003249305223286936
move 0.001953125 dirt from (44.23275429365532, 21.746521831360468, 41.790128382243196) to (43.72732187749419, 21.299210422187166, 41.80536614211598) for a cost of 0.0013185859374320451
move 0.001953125 dirt from (25.023999597382907, 40.99049129250478, 25.158880477078668) to (23.33834153273026, 42.20247592792429, 24.41696991731962) for a cost of 0.004306089704922833
move 0.001953125 dirt from (46.93912601756428, 15.961427122799172, 21.52480838663421) to (39.21570357400755, 17.7789885504786, 21.517208899431317) for a cost of 0.015496892006783973
move 0.001953125 dirt from (26.701871492659688, 44.04215323653604, 31.095204448494535) to (26.76976489786442, 43.84297886046283, 31.634133151554423) for a cost of 0.0011299872053928636
move 0.001953125 dirt from (21.061745844389666, 21.524559127841833, 27.39893350

100%|██████████| 511/511 [00:00<00:00, 28730.04it/s]


move 0.001953125 dirt from (15.29618203584488, 51.571372665003516, 31.167914178071175) to (15.251731122282353, 51.54110757878445, 30.574599063184053) for a cost of 0.0011635686818182727
move 0.001953125 dirt from (32.695958518493136, 32.04079908387777, 22.99876533034505) to (34.181624856150336, 30.737535898139747, 22.480619873101094) for a cost of 0.003990389777345464
move 0.001953125 dirt from (18.53251979335952, 53.47342074460279, 28.15605041670652) to (16.682142360510987, 53.420659686537526, 27.86661364179713) for a cost of 0.00365941516514119
move 0.001953125 dirt from (26.42012040136039, 14.805475271872309, 33.68440351390133) to (28.932390022176786, 15.021196887161306, 31.400942128300983) for a cost of 0.006644136928227833
move 0.001953125 dirt from (13.034357827686168, 50.17805753685383, 33.961374922036114) to (11.987823658434579, 48.60143275220088, 34.45267134899738) for a cost of 0.0038185225439371353
move 0.001953125 dirt from (27.929253750772865, 8.183989567307538, 22.6656154

100%|██████████| 511/511 [00:00<00:00, 33602.83it/s]


move 0.001953125 dirt from (45.229690666373955, 15.887209740330256, 21.91162644783467) to (44.61487659082789, 16.580414302423492, 21.90216584953963) for a cost of 0.0018097981437506403
move 0.001953125 dirt from (25.12183765168256, 11.070681220684436, 28.724056533391092) to (26.33001936322957, 10.919142516193471, 34.04702549292436) for a cost of 0.010664968477432401
move 0.001953125 dirt from (15.22106686668917, 40.62754059676768, 22.48893699052654) to (18.445651614568597, 40.40165446592351, 22.696060252743834) for a cost of 0.0063263981132952365
move 0.001953125 dirt from (36.889261408624165, 16.97223655559095, 21.852074900881682) to (36.53960780236284, 17.397001073202134, 21.579858705970224) for a cost of 0.0011988818308605504
move 0.001953125 dirt from (47.219945048222385, 32.17324464096677, 25.81499083482393) to (46.5255378455423, 32.92745658985062, 26.49463927925757) for a cost of 0.002402390620568411
move 0.001953125 dirt from (34.263329898460015, 13.421546855605156, 30.855375048